In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 
pd.set_option('display.max_columns', None)

In [2]:
data = pd.read_csv('learningSet.csv')

C:\Users\nolwe\AppData\Local\Temp\ipykernel_35348\3673836359.py:1: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('learningSet.csv')


In [3]:
data.shape

(95412, 481)

# Separating target, categorical and numerical columns

In [4]:
Y = data[['TARGET_B', 'TARGET_D']]

numerical = data.select_dtypes(np.number)
numerical = numerical.drop(columns = ['TARGET_B', 'TARGET_D'])

categorical = data.select_dtypes(object)


# Cleaning categorical columns

In [5]:
categorical.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 95412 entries, 0 to 95411
Data columns (total 74 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   OSOURCE   95412 non-null  object
 1   STATE     95412 non-null  object
 2   ZIP       95412 non-null  object
 3   MAILCODE  95412 non-null  object
 4   PVASTATE  95412 non-null  object
 5   NOEXCH    95412 non-null  object
 6   RECINHSE  95412 non-null  object
 7   RECP3     95412 non-null  object
 8   RECPGVG   95412 non-null  object
 9   RECSWEEP  95412 non-null  object
 10  MDMAUD    95412 non-null  object
 11  DOMAIN    95412 non-null  object
 12  CLUSTER   95412 non-null  object
 13  AGEFLAG   95412 non-null  object
 14  HOMEOWNR  95412 non-null  object
 15  CHILD03   95412 non-null  object
 16  CHILD07   95412 non-null  object
 17  CHILD12   95412 non-null  object
 18  CHILD18   95412 non-null  object
 19  GENDER    95412 non-null  object
 20  DATASRCE  95412 non-null  object
 21  SOLP3     95

## Checking for null values to identify column candidates for removal

In [6]:
#looking at the percentage of null values in each column
cat_nulls=pd.DataFrame(categorical.isna().sum()*100/len(categorical)).reset_index()
cat_nulls.columns = ['column_name', 'nulls_percentage']
cat_nulls

,column_name,nulls_percentage
0,OSOURCE,0.000000
1,STATE,0.000000
2,ZIP,0.000000
3,MAILCODE,0.000000
4,PVASTATE,0.000000
...,...,...
69,RFA_2A,0.000000
70,MDMAUD_R,0.000000
71,MDMAUD_F,0.000000
72,MDMAUD_A,0.000000


Replacing empty values with NaN or another valid value

In [7]:
categorical['MAILCODE'] = categorical['MAILCODE'].apply(lambda x: x.replace(" ", "A"))
categorical = categorical.apply(lambda x: x.replace(" ", np.NaN))

#checking again for null values
cat_nulls=pd.DataFrame(categorical.isna().sum()*100/len(categorical)).reset_index()
cat_nulls.columns = ['column_name', 'nulls_percentage']
cat_nulls

,column_name,nulls_percentage
0,OSOURCE,0.972624
1,STATE,0.000000
2,ZIP,0.000000
3,MAILCODE,0.000000
4,PVASTATE,98.471890
...,...,...
69,RFA_2A,0.000000
70,MDMAUD_R,0.000000
71,MDMAUD_F,0.000000
72,MDMAUD_A,0.000000


'OSOURCE' indicates which mailing list the donor was originally acquired from, the symbol's signification is not provided making this column not useable. Since we'll keep the state, the ZIP column is redondant and can be dropped. 

In [8]:
#creating a list containing the names of the columns to drop
drop_list = ['OSOURCE', 'ZIP']

Subsetting columns with over 50% of missing values and appending them the list of columns to drop

In [9]:
columns_above_threshold = cat_nulls[cat_nulls['nulls_percentage']>50]
len(columns_above_threshold['column_name'])
#there is 36 columns with more than 50% percent missing values

36

In [10]:
#appending the drop_list with the column names
drop_list = drop_list + list(columns_above_threshold['column_name'])
print(drop_list)

['OSOURCE', 'ZIP', 'PVASTATE', 'RECINHSE', 'RECP3', 'RECPGVG', 'RECSWEEP', 'CHILD03', 'CHILD07', 'CHILD12', 'CHILD18', 'SOLP3', 'SOLIH', 'MAJOR', 'GEOCODE', 'COLLECT1', 'VETERANS', 'BIBLE', 'CATLG', 'HOMEE', 'PETS', 'CDPLAY', 'STEREO', 'PCOWNERS', 'PHOTO', 'CRAFTS', 'FISHER', 'GARDENIN', 'BOATS', 'WALKER', 'KIDSTUFF', 'CARDS', 'PLATES', 'LIFESRC', 'PEPSTRFL', 'RFA_15', 'RFA_20', 'RFA_23']


In [11]:
#looking at the values for each column and their signification using the features description
for items in drop_list:
    display(categorical[items].value_counts(dropna=False))

#can be dropped: OSOURCE, ZIP, PVASTATE, CHILD03, CHILD07, CHILD012, CHILD18, SOLP3, MAJOR, GEOCODE, LIFESRC, RFA_15, RFA_20, RFA_23
#transform NaN to Not : RECINHSE, RECP3, RECPGVG, RECSWEEP, PEPSTRFL
#transform NaN to Possible: SOLIH
#transform NaN to N : 'COLLECT1', 'VETERANS', 'BIBLE', 'CATLG', 'HOMEE', 'PETS', 'CDPLAY', 'STEREO', 'PCOWNERS', 'PHOTO', 'CRAFTS', 'FISHER', 'GARDENIN', 'BOATS', 'WALKER', 'KIDSTUFF', 'CARDS', 'PLATES'


OSOURCE
MBC    4539
SYN    3563
AML    3430
BHG    3324
IMP    2986
       ... 
MDD       1
NRM       1
HDP       1
CRP       1
VIC       1
Name: count, Length: 896, dtype: int64

ZIP
85351     61
92653     59
85710     54
95608     50
60619     45
          ..
70058-     1
93024      1
96306      1
30060-     1
32776-     1
Name: count, Length: 19938, dtype: int64

PVASTATE
NaN    93954
P       1453
E          5
Name: count, dtype: int64

RECINHSE
NaN    88709
X       6703
Name: count, dtype: int64

RECP3
NaN    93395
X       2017
Name: count, dtype: int64

...

LIFESRC
NaN    54032
2      20027
3      11479
1       9874
Name: count, dtype: int64

PEPSTRFL
NaN    50143
X      45269
Name: count, dtype: int64

RFA_15
NaN    65477
A1G     3461
A2F     3084
S4E     2016
S4D     1965
S2F     1729
S3E     1631
A3E     1370
A2G     1361
S4F     1349
A3F     1249
S3D     1107
A4E     1037
S3F     1026
A3D      854
N2E      819
A4F      805
A4D      783
S2G      706
S4G      612
N2F      512
A3G      511
S3G      456
A4G      385
N2G      233
N3D      223
N3E      206
N3F      141
N4D       91
N3G       80
N4E       65
N4F       47
N4G       20
U1F        1
Name: count, dtype: int64

RFA_20
NaN    50200
A1E     6408
A1F     4947
A1D     3340
A1G     2193
       ...  
U1E        2
I1G        2
L4E        1
L3F        1
U1D        1
Name: count, Length: 80, dtype: int64

RFA_23
NaN    56274
A1F     4607
A1E     4348
A1G     2401
S2E     2243
       ...  
1E         1
N1E        1
2D         1
U1D        1
L1D        1
Name: count, Length: 87, dtype: int64

A lot of values were wrongly entered and considered as NaN. We need to fillna with a correct/valid value.

In [12]:
#editing the drop_list:
keep_list = ['RECINHSE', 'RECP3', 'RECPGVG', 'RECSWEEP', 'SOLIH', 'COLLECT1', 'VETERANS', 'BIBLE', 'CATLG', 'HOMEE', 'PETS', 'CDPLAY', 'STEREO', 'PCOWNERS', 'PHOTO', 'CRAFTS', 'FISHER', 'GARDENIN', 'BOATS', 'WALKER', 'KIDSTUFF', 'CARDS', 'PLATES', 'PEPSTRFL']
for items in keep_list:
    drop_list.remove(items)
print(drop_list)

['OSOURCE', 'ZIP', 'PVASTATE', 'CHILD03', 'CHILD07', 'CHILD12', 'CHILD18', 'SOLP3', 'MAJOR', 'GEOCODE', 'LIFESRC', 'RFA_15', 'RFA_20', 'RFA_23']


In [13]:
#transform NaN to Not:
list_not = ['RECINHSE', 'RECP3', 'RECPGVG', 'RECSWEEP', 'PEPSTRFL']
for items in list_not:
    categorical[items].fillna('Not', inplace=True)
    
#transform NaN to Possible:
categorical['SOLIH'].fillna('Possible', inplace = True)

#transform NaN to 'N':
list_N = ['COLLECT1', 'VETERANS', 'BIBLE', 'CATLG', 'HOMEE', 'PETS', 'CDPLAY', 'STEREO', 'PCOWNERS', 'PHOTO', 'CRAFTS', 'FISHER', 'GARDENIN', 'BOATS', 'WALKER', 'KIDSTUFF', 'CARDS', 'PLATES']
for items in list_N:
    categorical[items].fillna('N', inplace=True)

Dropping the columns from the drop_list

In [14]:
categorical.drop(drop_list, axis =1, inplace=True)
categorical

,STATE,MAILCODE,NOEXCH,RECINHSE,RECP3,RECPGVG,RECSWEEP,MDMAUD,DOMAIN,CLUSTER,AGEFLAG,HOMEOWNR,GENDER,DATASRCE,SOLIH,COLLECT1,VETERANS,BIBLE,CATLG,HOMEE,PETS,CDPLAY,STEREO,PCOWNERS,PHOTO,CRAFTS,FISHER,GARDENIN,BOATS,WALKER,KIDSTUFF,CARDS,PLATES,PEPSTRFL,RFA_2,RFA_3,RFA_4,RFA_5,RFA_6,RFA_7,RFA_8,RFA_9,RFA_10,RFA_11,RFA_12,RFA_13,RFA_14,RFA_16,RFA_17,RFA_18,RFA_19,RFA_21,RFA_22,RFA_24,RFA_2R,RFA_2A,MDMAUD_R,MDMAUD_F,MDMAUD_A,GEOCODE2
0,IL,A,0,Not,Not,Not,Not,XXXX,T2,36,NaN,NaN,F,NaN,Possible,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,X,L4E,S4E,S4E,S4E,S4E,S4E,S4E,S4E,S4E,S4E,S4E,S4E,S4E,S4E,S4E,S4E,S4E,S4E,S4E,S4E,L,E,X,X,X,C
1,CA,A,0,Not,Not,Not,Not,XXXX,S1,14,E,H,M,3,Possible,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,Not,L2G,A2G,A2G,A2G,A2G,A1E,A1E,A1E,A1E,A1E,A1E,NaN,NaN,L1E,NaN,NaN,N1E,N1E,N1E,F1E,L,G,X,X,X,A
2,NC,A,0,Not,Not,Not,Not,XXXX,R2,43,NaN,U,M,3,Possible,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,X,L4E,S4E,S4E,S4E,S4E,S4F,S4F,S4F,NaN,S4F,S4F,S4F,S4F,S4F,NaN,S4D,S4D,NaN,S4D,S3D,L,E,X,X,X,C
3,CA,A,0,Not,Not,Not,Not,XXXX,R2,44,E,U,F,3,Possible,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,X,L4E,S4E,S4E,S4E,S4E,S4E,S4E,S4E,NaN,S4E,S4E,S4E,S4E,S4E,S2D,S2D,A1D,A1D,A1D,NaN,L,E,X,X,X,C
4,FL,A,0,X,X,Not,Not,XXXX,S2,16,E,H,F,3,12,N,N,Y,Y,N,N,N,Y,N,N,Y,N,Y,N,Y,N,Y,N,Not,L2F,A2F,A2F,A2F,A1D,I2D,A1E,A1E,L1D,A1E,A1E,L1D,L3D,L3D,A2D,A2D,A3D,A3D,I4E,A3D,L,F,X,X,X,A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95407,AK,A,0,Not,Not,Not,Not,XXXX,C2,27,NaN,NaN,M,NaN,Possible,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,Not,L1G,F1G,F1G,NaN,F1G,P1G,P1G,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,L,G,X,X,X,C
95408,TX,A,0,Not,Not,Not,Not,XXXX,C1,24,E,H,M,3,Possible,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,Not,L1F,NaN,NaN,NaN,NaN,P1F,P1F,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,L,F,X,X,X,A
95409,MI,A,0,Not,X,Not,Not,XXXX,C3,30,NaN,NaN,M,NaN,Possible,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,X,L3E,S4E,S4E,S3E,S3E,NaN,A2E,N3E,N3E,N3E,N3E,N3E,N3E,F1D,NaN,F1D,NaN,P1D,P1D,NaN,L,E,X,X,X,B
95410,CA,A,0,X,Not,Not,Not,XXXX,C1,24,E,H,F,2,12,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,X,L4F,S4F,A3F,S4F,S4F,S4F,S4F,S4F,S4F,S4F,S4F,S4F,S4F,S3F,S2F,S2F,A1F,A1F,A1F,S3F,L,F,X,X,X,A


## Reducing the number of categories

In [15]:
categorical.nunique()

STATE        57
MAILCODE      2
NOEXCH        5
RECINHSE      2
RECP3         2
RECPGVG       2
RECSWEEP      2
MDMAUD       28
DOMAIN       16
CLUSTER      53
AGEFLAG       2
HOMEOWNR      2
GENDER        6
DATASRCE      3
SOLIH         8
COLLECT1      2
VETERANS      2
BIBLE         2
CATLG         2
HOMEE         2
PETS          2
CDPLAY        2
STEREO        2
PCOWNERS      2
PHOTO         2
CRAFTS        2
FISHER        2
GARDENIN      2
BOATS         2
WALKER        2
KIDSTUFF      2
CARDS         2
PLATES        2
PEPSTRFL      2
RFA_2        14
RFA_3        70
RFA_4        63
RFA_5        40
RFA_6       108
RFA_7       105
RFA_8       108
RFA_9       106
RFA_10       93
RFA_11      100
RFA_12      106
RFA_13       86
RFA_14       94
RFA_16      122
RFA_17      117
RFA_18      121
RFA_19      107
RFA_21      101
RFA_22      116
RFA_24       96
RFA_2R        1
RFA_2A        4
MDMAUD_R      5
MDMAUD_F      4
MDMAUD_A      5
GEOCODE2      4
dtype: int64

I'll reduce the number of values to have approximately 10 categories maximum per column

In [16]:
cat_var = pd.DataFrame(categorical.nunique()).reset_index()
cat_var.columns = ['column_name', 'nunique']

In [17]:
above_10 = cat_var[cat_var['nunique']>10]
len(above_10['column_name'])
#there is 24 columns with more than 50% percent missing values

24

In [18]:
var_list = list(above_10['column_name'])
for items in var_list:
    display(categorical[items].value_counts(dropna= False))

STATE
CA    17343
FL     8376
TX     7535
IL     6420
MI     5654
NC     4160
WA     3577
GA     3403
IN     2980
WI     2795
MO     2712
TN     2492
AZ     2407
OR     2183
MN     2175
CO     2032
SC     1759
AL     1705
KY     1620
OK     1617
LA     1594
KS     1294
IA     1271
AR     1020
MS      986
NV      979
NM      877
NE      754
UT      569
ID      533
MT      527
HI      449
SD      301
WY      283
AK      282
ND      260
AP       81
NY       71
VA       55
OH       52
PA       38
MD       34
NJ       26
MA       25
CT       23
AA       18
AE       15
ME       11
NH        8
VT        7
RI        6
VI        5
WV        4
GU        4
DE        3
DC        1
AS        1
Name: count, dtype: int64

MDMAUD
XXXX    95118
C1CM       65
L1CM       44
I1CM       37
D2CM       28
C2CM       24
D1CM       20
L2CM       15
L1LM        8
C1LM        8
I2CM        7
D5CM        5
D5MM        5
D2MM        4
C5CM        3
C2MM        3
C2LM        3
D5TM        3
I1LM        3
I5CM        1
C1MM        1
I5MM        1
C5MM        1
I2MM        1
L1MM        1
L2LM        1
C5TM        1
L2TM        1
Name: count, dtype: int64

DOMAIN
R2     13623
T2     12369
S1     11503
S2      8530
C2      8264
C1      6145
C3      5280
T1      4982
R3      4809
U1      4510
U2      3254
U3      2598
NaN     2316
T3      2176
S3      1891
U4      1804
R1      1358
Name: count, dtype: int64

CLUSTER
40     3979
35     3624
36     3617
27     3553
24     3539
49     3315
12     3000
18     2897
13     2706
30     2607
39     2601
45     2515
43     2385
11     2378
51     2322
NaN    2316
14     2246
41     2176
44     1953
16     1947
2      1942
21     1891
8      1828
10     1804
46     1800
28     1630
17     1621
20     1568
53     1441
3      1428
34     1358
42     1358
23     1324
31     1285
22     1282
25     1275
38     1200
15     1173
1      1140
7      1076
5      1014
37      972
26      947
47      868
29      859
32      795
48      787
50      776
9       770
6       603
33      593
4       561
19      497
52      270
Name: count, dtype: int64

RFA_2
L1F    30380
L1G    12384
L2F    10961
L3E     7767
L2E     4989
L4D     4914
L1E     4911
L2G     4595
L4E     3972
L3F     3523
L3D     2498
L4F     2100
L3G     1503
L4G      915
Name: count, dtype: int64

...

RFA_18
NaN    21263
A1E     7186
A1F     5510
A1D     3941
A1G     2408
       ...  
P1B        1
S2A        1
S3A        1
A4A        1
N3B        1
Name: count, Length: 122, dtype: int64

RFA_19
NaN    24492
A1E     7248
A1F     5347
A1D     4156
A1G     2330
       ...  
A4B        3
P1B        2
S4A        2
L4C        1
A4A        1
Name: count, Length: 108, dtype: int64

RFA_21
NaN    35212
A1E     6729
A1F     5127
A1D     3607
A1G     2245
       ...  
A4B        2
S3A        2
P1B        2
S4A        2
A4A        1
Name: count, Length: 102, dtype: int64

RFA_22
NaN    25648
A1E     7233
A1F     5324
A1D     4113
A1G     2318
       ...  
S4A        2
A1A        1
F1B        1
L4C        1
A4A        1
Name: count, Length: 117, dtype: int64

RFA_24
NaN    36973
A1E     7227
A1F     5032
A1D     4563
F1D     3315
       ...  
N1C        2
U1C        2
A1C        2
U1G        1
L4C        1
Name: count, Length: 97, dtype: int64

I've decided to drop the RFA columns as this status is already summed up in RFA_2A, RFA_2R and RFA_2F (the latter is in the numerical columns)

In [19]:
#dropping RFA columns
rfa_list = ['RFA_2', 'RFA_3', 'RFA_4', 'RFA_5', 'RFA_6', 'RFA_7', 'RFA_8', 'RFA_9', 'RFA_10', 'RFA_11', 'RFA_12', 'RFA_13', 'RFA_14', 'RFA_16', 'RFA_17', 'RFA_18', 'RFA_19', 'RFA_21', 'RFA_22', 'RFA_24']
categorical.drop(rfa_list, axis=1, inplace = True)

In [20]:
#editing and looking at the remaining columns that need a reduced number of categories
for items in rfa_list:
    var_list.remove(items)
    
for items in var_list:
    display(categorical[items].value_counts(dropna= False))

STATE
CA    17343
FL     8376
TX     7535
IL     6420
MI     5654
NC     4160
WA     3577
GA     3403
IN     2980
WI     2795
MO     2712
TN     2492
AZ     2407
OR     2183
MN     2175
CO     2032
SC     1759
AL     1705
KY     1620
OK     1617
LA     1594
KS     1294
IA     1271
AR     1020
MS      986
NV      979
NM      877
NE      754
UT      569
ID      533
MT      527
HI      449
SD      301
WY      283
AK      282
ND      260
AP       81
NY       71
VA       55
OH       52
PA       38
MD       34
NJ       26
MA       25
CT       23
AA       18
AE       15
ME       11
NH        8
VT        7
RI        6
VI        5
WV        4
GU        4
DE        3
DC        1
AS        1
Name: count, dtype: int64

MDMAUD
XXXX    95118
C1CM       65
L1CM       44
I1CM       37
D2CM       28
C2CM       24
D1CM       20
L2CM       15
L1LM        8
C1LM        8
I2CM        7
D5CM        5
D5MM        5
D2MM        4
C5CM        3
C2MM        3
C2LM        3
D5TM        3
I1LM        3
I5CM        1
C1MM        1
I5MM        1
C5MM        1
I2MM        1
L1MM        1
L2LM        1
C5TM        1
L2TM        1
Name: count, dtype: int64

DOMAIN
R2     13623
T2     12369
S1     11503
S2      8530
C2      8264
C1      6145
C3      5280
T1      4982
R3      4809
U1      4510
U2      3254
U3      2598
NaN     2316
T3      2176
S3      1891
U4      1804
R1      1358
Name: count, dtype: int64

CLUSTER
40     3979
35     3624
36     3617
27     3553
24     3539
49     3315
12     3000
18     2897
13     2706
30     2607
39     2601
45     2515
43     2385
11     2378
51     2322
NaN    2316
14     2246
41     2176
44     1953
16     1947
2      1942
21     1891
8      1828
10     1804
46     1800
28     1630
17     1621
20     1568
53     1441
3      1428
34     1358
42     1358
23     1324
31     1285
22     1282
25     1275
38     1200
15     1173
1      1140
7      1076
5      1014
37      972
26      947
47      868
29      859
32      795
48      787
50      776
9       770
6       603
33      593
4       561
19      497
52      270
Name: count, dtype: int64

In [21]:
#for STATE, I'll combine the states with a frequency less than 2500 in others
df = pd.DataFrame(categorical['STATE'].value_counts()).reset_index()

df.columns = ['state', 'count']

#setting a cutoff point at 2500 and creating a list of states<2500
other_states = list(df[df['count']<2500]['state'])

def clean_state(x):
    if x in other_states:
        return 'other'
    else:
        return x
    
categorical['STATE'] = list(map(clean_state, categorical['STATE']))

In [22]:
#for domain I'll fillna with X5 (a non-attributed value that would represent 'unknown' while keeping the same format)
categorical['DOMAIN'] = categorical['DOMAIN'].fillna('X5')

#I'll extract the information in 2 columns: Urbanicity and socio-economic_status and drop DOMAIN
categorical['URBANICITY'] = list(map(lambda x: x[0], categorical['DOMAIN']))
categorical['SES'] = list(map(lambda x: x[1], categorical['DOMAIN']))
categorical.drop('DOMAIN', axis=1, inplace=True)

In [23]:
#for MDMAUD, I've decided to drop this column as well as the other MDMAUD_X columns since they are really imbalanced and would not contribute much to the caracterization of our targets

mdmaud_list=['MDMAUD', 'MDMAUD_R', 'MDMAUD_F', 'MDMAUD_A']

for items in mdmaud_list:
    categorical.drop(items, axis=1, inplace=True)

In [24]:
#for 'CLUSTER' I'll fillna with the most represented value and set it as a number to reduce the number of values to encode later

categorical['CLUSTER'] = categorical['CLUSTER'].fillna('40')
categorical['CLUSTER'] = categorical['CLUSTER'].astype('int64')

## Checking for null values again to fill them with valid values this time

In [25]:
cat_nulls=pd.DataFrame(categorical.isna().sum()*100/len(categorical)).reset_index()
cat_nulls.columns = ['column_name', 'nulls_percentage']
columns_with_na = cat_nulls[cat_nulls['nulls_percentage']>0]
columns_with_na

,column_name,nulls_percentage
2,NOEXCH,0.007337
8,AGEFLAG,30.968851
9,HOMEOWNR,23.296860
10,GENDER,3.099191
11,DATASRCE,22.303274
34,GEOCODE2,0.334339


In [26]:
#for HOMEOWNR 

#checking the values
categorical['HOMEOWNR'].value_counts(dropna=False)

#filling NaN with Unknown after having looked at the documentation
categorical['HOMEOWNR'] = categorical['HOMEOWNR'].fillna('Unknown')

In [27]:
#for GENDER

#checking the values
categorical['GENDER'].value_counts(dropna=False)

#filling NaN with F (most represented value) and grouping the other values in 'other'
categorical['GENDER'] = categorical['GENDER'].fillna('F')

def clean_gender(x):
    if x=='F' or x=='M':
        return x
    else:
        return 'other'
    
categorical['GENDER'] = list(map(clean_gender, categorical['GENDER']))

In [28]:
#for DATASRC

#checking the values
categorical['DATASRCE'].value_counts(dropna=False)

#filling NaN with 0 after having looked at the documentation
categorical['DATASRCE'] = categorical['DATASRCE'].fillna('0')

In [29]:
#for GEOCODE2

#checking the values
categorical['GEOCODE2'].value_counts(dropna=False)

#filling NaN with A (most represented value)
categorical['GEOCODE2'] = categorical['GEOCODE2'].fillna('A')

In [30]:
#because NOEXCH is imbalanced and AGEFLAG has more than 25% null values I will drop those 2 columns 
#I'll also drop mailcode because of its imbalance

imbalance=['NOEXCH', 'MAILCODE', 'AGEFLAG']
categorical.drop(imbalance, axis=1, inplace =True)
categorical

,STATE,RECINHSE,RECP3,RECPGVG,RECSWEEP,CLUSTER,HOMEOWNR,GENDER,DATASRCE,SOLIH,COLLECT1,VETERANS,BIBLE,CATLG,HOMEE,PETS,CDPLAY,STEREO,PCOWNERS,PHOTO,CRAFTS,FISHER,GARDENIN,BOATS,WALKER,KIDSTUFF,CARDS,PLATES,PEPSTRFL,RFA_2R,RFA_2A,GEOCODE2,URBANICITY,SES
0,IL,Not,Not,Not,Not,36,Unknown,F,0,Possible,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,X,L,E,C,T,2
1,CA,Not,Not,Not,Not,14,H,M,3,Possible,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,Not,L,G,A,S,1
2,NC,Not,Not,Not,Not,43,U,M,3,Possible,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,X,L,E,C,R,2
3,CA,Not,Not,Not,Not,44,U,F,3,Possible,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,X,L,E,C,R,2
4,FL,X,X,Not,Not,16,H,F,3,12,N,N,Y,Y,N,N,N,Y,N,N,Y,N,Y,N,Y,N,Y,N,Not,L,F,A,S,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95407,other,Not,Not,Not,Not,27,Unknown,M,0,Possible,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,Not,L,G,C,C,2
95408,TX,Not,Not,Not,Not,24,H,M,3,Possible,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,Not,L,F,A,C,1
95409,MI,Not,X,Not,Not,30,Unknown,M,0,Possible,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,X,L,E,B,C,3
95410,CA,X,Not,Not,Not,24,H,F,2,12,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,X,L,F,A,C,1


# Cleaning numerical columns

In [31]:
numerical.shape

(95412, 405)

## Checking for null values to identify column candidates for removal

In [32]:
#creating df with the percentage of null values for each column
num_nulls=pd.DataFrame(numerical.isna().sum()*100/len(numerical)).reset_index()
num_nulls.columns = ['column_name', 'nulls_percentage']

#there is a lot more columns than for the categoricals so I'll set a threshold at 25% (to keep less columns)
columns_above_threshold = num_nulls[num_nulls['nulls_percentage']>25]
len(columns_above_threshold['column_name'])
#there is 72 columns with more than 25% percent missing values

72

In [33]:
drop_list = list(columns_above_threshold['column_name'])
drop_list
#I'll keep 'WEALTH_X'
drop_list.remove('WEALTH1')
drop_list.remove('WEALTH2')

#after looking at the meaning of each columns I have decided to drop the rest of them
numerical.drop(drop_list, axis=1, inplace= True)
numerical.shape

(95412, 335)

## Cleaning the remaining columns containing NaN  values

In [34]:
df = pd.DataFrame(numerical.isna().sum()).reset_index()
df.columns = ['column_name', 'nulls']
df[df['nulls']>0]

,column_name,nulls
3,AGE,23665
4,INCOME,21286
5,WEALTH1,44732
14,WEALTH2,43823
135,MSA,132
136,ADI,132
137,DMA,132
302,ADATE_3,1950
303,ADATE_4,2191
304,ADATE_6,3557


In [35]:
#grouping columns containing "ADATE" in a list and dropping them
drop_list=[]

for col_name in numerical.columns:
        if "ADATE" in col_name:
            drop_list.append(col_name)
            
numerical.drop(drop_list, axis=1, inplace=True)

In [36]:
#for WEALTHX, AGE, INCOME and CLUSTER2, I'll fillna with the mean
#I won't round the mean to integers so that those NaN values can still be distinct from the other categories 
#By doing that I won't to avoid creating a bias for one of the categories

mean_list=['WEALTH1', 'WEALTH2', 'AGE', 'INCOME', 'CLUSTER2']
for items in mean_list:
    numerical[items] = numerical[items].fillna(round(np.mean(numerical[items]),2))

In [37]:
#we're left with MSA, ADI, DMA, NEXTDATE and TIMELAG
num_with_na = ['MSA', 'ADI', 'DMA', 'NEXTDATE', 'TIMELAG']
for items in num_with_na:
    display(numerical[items].value_counts(dropna=False))

MSA
0.0       21333
4480.0     4606
1600.0     4059
2160.0     2586
520.0      1685
          ...  
9140.0        1
3200.0        1
9280.0        1
743.0         1
8480.0        1
Name: count, Length: 299, dtype: int64

ADI
13.0     7296
51.0     4622
65.0     3765
57.0     2836
105.0    2617
         ... 
651.0       1
103.0       1
601.0       1
161.0       1
147.0       1
Name: count, Length: 205, dtype: int64

DMA
803.0    7296
602.0    4632
807.0    3765
505.0    2839
819.0    2588
         ... 
569.0       1
554.0       1
584.0       1
552.0       1
516.0       1
Name: count, Length: 207, dtype: int64

NEXTDATE
NaN       9973
9504.0    2253
9412.0    1970
8703.0    1959
9512.0    1870
          ... 
8107.0       1
7408.0       1
8207.0       1
8104.0       1
8412.0       1
Name: count, Length: 189, dtype: int64

TIMELAG
NaN       9973
5.0       8584
4.0       8490
3.0       8485
6.0       6648
          ... 
89.0         1
1044.0       1
53.0         1
442.0        1
1088.0       1
Name: count, Length: 69, dtype: int64

After looking at the meaning of the values, I've decided to replace NaN with the mode for MSA, ADI, DMA and 0 for NEXTDATE and TIMELAG

In [38]:
#replacing NaN with the mode 
numerical['MSA'] = numerical['MSA'].fillna(0.0)
numerical['ADI'] = numerical['ADI'].fillna(13.0)
numerical['DMA'] = numerical['DMA'].fillna(803.0)

#replacing NaN with 0
numerical['NEXTDATE'] = numerical['NEXTDATE'].fillna(0.0)
numerical['TIMELAG'] = numerical['TIMELAG'].fillna(0.0)

In [39]:
#checking if there are Nan values remaining
df = pd.DataFrame(numerical.isna().sum()).reset_index()
df.columns = ['column_name', 'nulls']
df[df['nulls']>0]

,column_name,nulls


In [40]:
numerical.shape

(95412, 322)

# Concatenating our dfs into a cleaned df

In [41]:
final_df = pd.concat([Y, numerical, categorical], axis=1)
final_df

,TARGET_B,TARGET_D,ODATEDW,TCODE,DOB,AGE,INCOME,WEALTH1,HIT,MALEMILI,MALEVET,VIETVETS,WWIIVETS,LOCALGOV,STATEGOV,FEDGOV,WEALTH2,POP901,POP902,POP903,POP90C1,POP90C2,POP90C3,POP90C4,POP90C5,ETH1,ETH2,ETH3,ETH4,ETH5,ETH6,ETH7,ETH8,ETH9,ETH10,ETH11,ETH12,ETH13,ETH14,ETH15,ETH16,AGE901,AGE902,AGE903,AGE904,AGE905,AGE906,AGE907,CHIL1,CHIL2,CHIL3,AGEC1,AGEC2,AGEC3,AGEC4,AGEC5,AGEC6,AGEC7,CHILC1,CHILC2,CHILC3,CHILC4,CHILC5,HHAGE1,HHAGE2,HHAGE3,HHN1,HHN2,HHN3,HHN4,HHN5,HHN6,MARR1,MARR2,MARR3,MARR4,HHP1,HHP2,DW1,DW2,DW3,DW4,DW5,DW6,DW7,DW8,DW9,HV1,HV2,HV3,HV4,HU1,HU2,HU3,HU4,HU5,HHD1,HHD2,HHD3,HHD4,HHD5,HHD6,HHD7,HHD8,HHD9,HHD10,HHD11,HHD12,ETHC1,ETHC2,ETHC3,ETHC4,ETHC5,ETHC6,HVP1,HVP2,HVP3,HVP4,HVP5,HVP6,HUR1,HUR2,RHP1,RHP2,RHP3,RHP4,HUPA1,HUPA2,HUPA3,HUPA4,HUPA5,HUPA6,HUPA7,RP1,RP2,RP3,RP4,MSA,ADI,DMA,IC1,IC2,IC3,IC4,IC5,IC6,IC7,IC8,IC9,IC10,IC11,IC12,IC13,IC14,IC15,IC16,IC17,IC18,IC19,IC20,IC21,IC22,IC23,HHAS1,HHAS2,HHAS3,HHAS4,MC1,MC2,MC3,TPE1,TPE2,TPE3,TPE4,TPE5,TPE6,TPE7,TPE8,TPE9,PEC1,PEC2,TPE10,TPE11,TPE12,TPE13,LFC1,LFC2,LFC3,LFC4,LFC5,LFC6,LFC7,LFC8,LFC9,LFC10,OCC1,OCC2,OCC3,OCC4,OCC5,OCC6,OCC7,OCC8,OCC9,OCC10,OCC11,OCC12,OCC13,EIC1,EIC2,EIC3,EIC4,EIC5,EIC6,EIC7,EIC8,EIC9,EIC10,EIC11,EIC12,EIC13,EIC14,EIC15,EIC16,OEDC1,OEDC2,OEDC3,OEDC4,OEDC5,OEDC6,OEDC7,EC1,EC2,EC3,EC4,EC5,EC6,EC7,EC8,SEC1,SEC2,SEC3,SEC4,SEC5,AFC1,AFC2,AFC3,AFC4,AFC5,AFC6,VC1,VC2,VC3,VC4,ANC1,ANC2,ANC3,ANC4,ANC5,ANC6,ANC7,ANC8,ANC9,ANC10,ANC11,ANC12,ANC13,ANC14,ANC15,POBC1,POBC2,LSC1,LSC2,LSC3,LSC4,VOC1,VOC2,VOC3,HC1,HC2,HC3,HC4,HC5,HC6,HC7,HC8,HC9,HC10,HC11,HC12,HC13,HC14,HC15,HC16,HC17,HC18,HC19,HC20,HC21,MHUC1,MHUC2,AC1,AC2,CARDPROM,NUMPROM,CARDPM12,NUMPRM12,RAMNTALL,NGIFTALL,CARDGIFT,MINRAMNT,MINRDATE,MAXRAMNT,MAXRDATE,LASTGIFT,LASTDATE,FISTDATE,NEXTDATE,TIMELAG,AVGGIFT,CONTROLN,HPHONE_D,RFA_2F,CLUSTER2,STATE,RECINHSE,RECP3,RECPGVG,RECSWEEP,CLUSTER,HOMEOWNR,GENDER,DATASRCE,SOLIH,COLLECT1,VETERANS,BIBLE,CATLG,HOMEE,PETS,CDPLAY,STEREO,PCOWNERS,PHOTO,CRAFTS,FISHER,GARDENIN,BOATS,WALKER,KIDSTUFF,CARDS,PLATES,PEPSTRFL,RFA_2R,RFA_2A,GEOCODE2,URBANICITY,SES
0,0,0.0,8901,0,3712,60.00,3.89,5.35,0,0,39,34,18,10,2,1,5.00,992,264,332,0,35,65,47,53,92,1,0,0,11,0,0,0,0,0,0,0,11,0,0,0,39,48,51,40,50,54,25,31,42,27,11,14,18,17,13,11,15,12,11,34,25,18,26,10,23,18,33,49,28,12,4,61,7,12,19,198,276,97,95,2,2,0,0,7,7,0,479,635,3,2,86,14,96,4,7,38,80,70,32,84,16,6,2,5,9,15,3,17,50,25,0,0,0,2,7,13,27,47,0,1,61,58,61,15,4,2,0,0,14,1,0,0,2,5,17,73,0.0,177.0,682.0,307,318,349,378,12883,13,23,23,23,15,1,0,0,1,4,25,24,26,17,2,0,0,2,28,4,51,1,46,54,3,88,8,0,0,0,0,0,0,4,1,13,14,16,2,45,56,64,50,64,44,62,53,99,0,0,9,3,8,13,9,0,3,9,3,15,19,5,4,3,0,3,41,1,0,7,13,6,5,0,4,9,4,1,3,10,2,1,7,78,2,0,120,16,10,39,21,8,4,3,5,20,3,19,4,0,0,0,18,39,0,34,23,18,16,1,4,0,23,0,0,5,1,0,0,0,0,0,2,0,3,74,88,8,0,4,96,77,19,13,31,5,14,14,31,54,46,0,0,90,0,10,0,0,0,33,65,40,99,99,6,2,10,7,27,74,6,14,240.0,31,14,5.0,9208,12.0,9402,10.0,9512,8911,9003.0,4.0,7.741935,95515,0,4,39.0,IL,Not,Not,Not,Not,36,Unknown,F,0,Possible,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,X,L,E,C,T,2
1,0,0.0,9401,1,5202,46.00,6.00,9.00,16,0,15,55,11,6,2,1,9.00,3611,940,998,99,0,0,50,50,67,0,0,31,6,4,2,6,4,14,0,0,2,0,1,4,34,41,43,32,42,45,32,33,46,21,13,14,33,23,10,4,2,11,16,36,22,15,12,1,5,4,21,75,55,23,9,69,4,3,24,317,360,99,99,0,0,0,0,0,0,0,5468,5218,12,10,96,4,97,3,9,59,94,88,55,95,5,4,1,3,5,4,2,18,44,5,0,0,0,97,98,98,98,99,94,0,83,76,73,21,5,0,0,0,4,0,0,0,91,91,91,94,4480.0,13.0,803.0,1088,1096,1026,1037,36175,2,6,2,5,15,14,13,10,33,2,5,2,5,15,14,14,10,32,6,2,66,3,56,44,9,80,14,0,0,0,0,0,0,6,0,2,24,32,12,71,70,83,58,81,57,64,57,99,99,0,22,24,4,21,13,2,1,6,0,4,1,0,3,1,0,6,13,1,2,8,18,11,4,3,4,10,7,11,1,6,2,1,16,69,5,2,160,5,5,12,21,7,30,20,14,24,4,24,10,0,0,0,8,15,0,55,10,11,0,0,2,0,3,1,1,2,3,1,1,0,3,0,0,0,42,39,50,7,27,16,99,92,53,5,10,2,26,56,97,99,0,0,0,96,0,4,0,0,0,99,0,99,99,99,20,4,6,5,12,32,6,13,47.0,3,1,10.0,9310,25.0,9512,25.0,9512,9310,9504.0,18.0,15.666667,148535,0,2,1.0,CA,Not,Not,Not,Not,14,H,M,3,Possible,N,N,N,N,N,N,N,N,N

In [42]:
#verifying that there is no null values remaining
final_df.columns[final_df.isna().any()].tolist()

[]

In [43]:
final_df.to_csv('cleaned_data.csv', index=False)